In [1]:
import tkinter as tk
from tkinter import ttk
import time
import random
import threading
import redis
from PIL import Image, ImageTk  # Para manejar imágenes

# Configuración de Redis
r = redis.Redis(
    host='localhost',
    port=6379,
    db=0)

# Simulación de los jugadores
jugadores = [
    {"id": i + 1, "nombre": f"Jugador_{random.randint(1000, 9999)}", "puntos": random.randint(1000, 2000)}
    for i in range(50)
]

# Actualiza los puntajes de los jugadores
def actualizar_puntos(jugador_nombre):
    puntos_actuales = r.zscore("ranking_jugadores", jugador_nombre)
    if puntos_actuales is None:
        puntos_actuales = 0    

    nuevos_puntos = random.randint(100, 6000)
    puntos_finales = max(puntos_actuales + nuevos_puntos, 0)
    timestamp = time.time()

    # Actualizar puntajes en Redis
    r.zadd("ranking_jugadores", {jugador_nombre: puntos_finales})
    r.hset(f"{jugador_nombre}_last_update", "timestamp", timestamp)
    
    # Guardar en el historial
    historial_entrada = f"{time.ctime(timestamp)}: {puntos_finales}"
    r.lpush(f"{jugador_nombre}_historial", historial_entrada)

# Función para mostrar el top 3 jugadores en la interfaz
def mostrar_top3_ranking():
    ranking = r.zrevrange("ranking_jugadores", 0, 2, withscores=True)  # Top 3 jugadores
    return [(nombre.decode('utf-8'), puntos) for nombre, puntos in ranking]

# Hilo para actualizar puntajes en segundo plano
def actualizar_puntajes_periodicamente():
    while True:
        jugador_nombre = random.choice([jugador['nombre'] for jugador in jugadores])
        actualizar_puntos(jugador_nombre)
        time.sleep(10)
# Hilo para actualizar la interfaz gráfica
def actualizar_interfaz():
    while True:
        top3 = mostrar_top3_ranking()
        for i, (nombre, puntos) in enumerate(top3):
            etiquetas[i].config(text=f"{i+1}. {nombre}: {puntos} puntos")
        time.sleep(1)

# Configuración de la interfaz con Tkinter y ttk
def crear_interfaz():
    ventana = tk.Tk()
    ventana.title("Ranking de Jugadores")
    ventana.geometry("400x350")
    ventana.configure(bg="#1a1a1a")  # Fondo oscuro

    # Usar ttk para widgets modernos
    style = ttk.Style()
    style.theme_use("clam")  # Estilo moderno
    style.configure("TLabel", foreground="#ecf0f1", background="#1a1a1a", font=("Helvetica", 14))

    # Título estilizado
    titulo = ttk.Label(ventana, text="Top 3 Jugadores", font=("Helvetica", 20, "bold"))
    titulo.pack(pady=20)

    # Etiquetas para mostrar el top 3
    global etiquetas
    etiquetas = [ttk.Label(ventana, text="Cargando...", padding=10) for _ in range(3)]
    for etiqueta in etiquetas:
        etiqueta.pack(pady=5)

    # Botón para cerrar la ventana
    boton_cerrar = ttk.Button(ventana, text="Cerrar", command=ventana.destroy)
    boton_cerrar.pack(pady=20)

    # Hilos
    hilo_actualizar_puntajes = threading.Thread(target=actualizar_puntajes_periodicamente, daemon=True)
    hilo_actualizar_puntajes.start()

    hilo_interfaz = threading.Thread(target=actualizar_interfaz, daemon=True)
    hilo_interfaz.start()

    # Iniciar la interfaz gráfica
    ventana.mainloop()

# Ejecutar la interfaz
crear_interfaz()